# Import libraries

In [1]:
import yaml
import asyncio
import nest_asyncio
from acp_sdk.client import Client
from smolagents import LiteLLMModel
from utils.fastacp import AgentCollection, ACPCallingAgent
from colorama import Fore
from dotenv import load_dotenv

nest_asyncio.apply()

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

load_dotenv() 

True

# Test

## Insurance Agent

In [2]:
async def call_insurance_agent(input) -> None:
    async with Client(base_url="http://localhost:8001") as client:
        run = await client.run_sync(
            agent="insurance_agent", 
            input=input
        )

        print(Fore.YELLOW + run.output[0].parts[0].content + Fore.RESET)

In [3]:
query = "What is the waiting period for rehabilitation?"
asyncio.run(call_insurance_agent(input=query))

The waiting period for rehabilitation is 2 months, even if the condition is pre-existing. This waiting period also applies to palliative care and hospital psychiatric treatments.


## Health Agent

In [4]:
async def call_health_agent(input) -> None:
    async with Client(base_url="http://localhost:8000") as client:
        run = await client.run_sync(
            agent="health_agent", 
            input=input
        )
        
        print(Fore.LIGHTMAGENTA_EX + run.output[0].parts[0].content + Fore.RESET)

In [5]:
query2 = "Do I need rehabilitation after a shoulder reconstruction?"
asyncio.run(call_health_agent(input=query2))

Yes, rehabilitation is necessary and crucial after a shoulder reconstruction to ensure optimal recovery, restore mobility and strength, and prevent future injuries. It typically involves a structured program of physical therapy and exercises.


## Router Agent (Workflow)

In [6]:
model = LiteLLMModel(
    model_id=config["agents"]["router"]["llm_model"],
    max_tokens=config["agents"]["router"]["max_tokens"]
)

In [7]:
async def run_hospital_workflow(input) -> None:
    async with Client(base_url="http://localhost:8001") as insurer, Client(base_url="http://localhost:8000") as hospital:
        
        # Agents discovery
        agent_collection = await AgentCollection.from_acp(insurer, hospital)  
        acp_agents = {agent.name: {'agent':agent, 'client':client} for client, agent in agent_collection.agents}
        print(acp_agents) 

        # Passing the agents as tools to ACPCallingAgent
        acpagent = ACPCallingAgent(
            acp_agents=acp_agents, 
            model=model
        )
        
        # Running the agent with a user query
        result = await acpagent.run(input)
        print(Fore.YELLOW + f"Final result: {result}" + Fore.RESET)

In [8]:
query3 = "Do i need rehabilitation after a shoulder reconstruction and what is the waiting period from my insurance?"
asyncio.run(run_hospital_workflow(input=query3))

{'insurance_agent': {'agent': Agent(name='insurance_agent', description='This is an agent for questions around policy coverage. \nUse it to help answer questions on coverage and waiting periods.\nIt uses a RAG pattern to find answers based on policy documentation.', metadata=Metadata(annotations=None, documentation=None, license=None, programming_language=None, natural_languages=None, framework=None, capabilities=None, domains=None, tags=None, created_at=None, updated_at=None, author=None, contributors=None, links=None, dependencies=None, recommended_models=None)), 'client': <acp_sdk.client.client.Client object at 0x127fb3cb0>}, 'health_agent': {'agent': Agent(name='health_agent', description='This is a CodeAgent which supports the hospital to handle health-based questions for patients. \nCurrent or prospective patients can use it to find answers about their health and hospital treatments.', metadata=Metadata(annotations=None, documentation=None, license=None, programming_language=None